# Importações de Bibliotecas
Carrega as bibliotecas necessárias para manipulação de dados, modelagem de redes neurais, tokenização e processamento de linguagem natural.

In [1]:
import os
import json
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import BertTokenizer, BertModel
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from collections import deque


c:\Users\Pichau\Desktop\codes\desafio cientista de dados cnpq\chatbot_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Verificação do Dispositivo
Define o dispositivo (device) para usar GPU (cuda) ou CPU, dependendo da disponibilidade, garantindo uma execução mais rápida quando a GPU estiver disponível.

In [2]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')


# Configuração do Modelo (Config Class)
Define os hiperparâmetros e configurações do modelo, como o número de classes (intenção), épocas de treinamento, tamanho do lote, taxa de aprendizado e comprimento máximo dos tokens.

In [3]:
class Config:
    num_classes = 10  # Ajuste com base no número de intenções
    epochs = 15
    batch_size = 8
    lr = 1e-5
    max_length = 15

cfg = Config()


# Carregamento e Extração do Dataset JSON
Carrega o arquivo intent.json, que contém as intenções, padrões de texto e respostas. Extrai as primeiras dez intenções para limitar o escopo do modelo, organizando textos, labels e respostas.

In [4]:
with open("intent.json", "r") as f:
    data = json.load(f)

df_intents = data['intents'][:cfg.num_classes]  # Usar apenas as primeiras 10 intenções
texts = []
labels = []
responses = {}


# Mapeamento de Intenções para Labels
Cria dicionários (intent2label e label2intent) para mapear cada intenção para um índice numérico, facilitando o uso do modelo para classificação.

In [5]:
intent2label = {}
label2intent = {}
for idx, intent_data in enumerate(df_intents):
    intent = intent_data['intent']
    intent2label[intent] = idx
    label2intent[idx] = intent
    for text in intent_data['text']:
        texts.append(text)
        labels.append(idx)
    # Coletar respostas para cada intenção
    responses[intent] = intent_data['responses']


# Preparação dos Dados
Gera um DataFrame para visualizar e manipular dados, incluindo texto e seus respectivos rótulos.

In [6]:
import pandas as pd
df = pd.DataFrame({'text': texts, 'label': labels})


# Tokenização com BertTokenizer
Prepara o texto para o modelo BERT, convertendo-o em uma sequência de IDs com padding e truncamento, conforme o comprimento máximo definido.

In [7]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


# Definição da Classe IntentDataset
Define uma classe Dataset personalizada para carregar textos e labels tokenizados, garantindo compatibilidade com DataLoader do PyTorch.

In [8]:
class IntentDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels
        self.encodings = tokenizer(texts, padding='max_length', max_length=cfg.max_length, truncation=True, return_tensors="pt")

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item


# Divisão do Conjunto de Dados
Divide os dados em conjuntos de treinamento e validação, utilizando uma proporção de 80% para treino e 20% para validação.

In [9]:
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)


# Criação de Objetos Dataset
Cria os datasets de treino e validação para serem utilizados pelo DataLoader.

In [10]:
train_dataset = IntentDataset(train_texts, train_labels)
val_dataset = IntentDataset(val_texts, val_labels)


# Definição do Modelo BertClassifier
Define o modelo de classificação com BertModel, seguido por uma camada linear para identificar as intenções. Inclui Dropout para regularização e ReLU como função de ativação.

In [11]:
class BertClassifier(nn.Module):
    def __init__(self, dropout=0.2):
        super(BertClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, cfg.num_classes)
        self.relu = nn.ReLU()

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output  # Usar o pooler_output
        dropout_output = self.dropout(pooled_output)
        output = self.linear(dropout_output)
        return output


# Verificação e Carregamento do Modelo Salvo
Verifica se o modelo salvo (intent_model.pth) existe. Se existir, carrega o modelo, caso contrário, inicia o treinamento e salva o modelo para futuras execuções.

In [12]:
model_path = 'intent_model.pth'
model = BertClassifier()
if os.path.exists(model_path):
    print("Carregando modelo salvo...")
    model.load_state_dict(torch.load(model_path, map_location=device))
else:
    print("Modelo não encontrado. Iniciando o treinamento...")
    train(model, train_dataset, val_dataset, cfg.lr, cfg.epochs)
    torch.save(model.state_dict(), model_path)
    print("Modelo treinado e salvo.")

model.to(device)
model.eval()


Carregando modelo salvo...


C:\Users\Pichau\AppData\Local\Temp\ipykernel_12756\2633590380.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=d

BertClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

# Função de Treinamento (train)
Define a função de treinamento para calcular a perda e a precisão dos conjuntos de treino e validação. Utiliza CrossEntropyLoss e o otimizador Adam.

In [13]:
def train(model, train_dataset, val_dataset, learning_rate, epochs):
    train_loader = DataLoader(train_dataset, batch_size=cfg.batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=cfg.batch_size)

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    model.to(device)
    criterion.to(device)

    for epoch in range(epochs):
        model.train()
        total_acc_train = 0
        total_loss_train = 0

        for batch in train_loader:
            optimizer.zero_grad()
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask)
            loss = criterion(outputs, labels)
            total_loss_train += loss.item()
            acc = (outputs.argmax(dim=1) == labels).sum().item()
            total_acc_train += acc

            loss.backward()
            optimizer.step()

        model.eval()
        total_acc_val = 0
        total_loss_val = 0

        with torch.no_grad():
            for batch in val_loader:
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)

                outputs = model(input_ids, attention_mask)
                loss = criterion(outputs, labels)
                total_loss_val += loss.item()
                acc = (outputs.argmax(dim=1) == labels).sum().item()
                total_acc_val += acc

        print(f"Epoch {epoch + 1}/{epochs}:")
        print(f"Train Loss: {total_loss_train / len(train_dataset):.3f}, Train Accuracy: {total_acc_train / len(train_dataset):.3f}")
        print(f"Val Loss: {total_loss_val / len(val_dataset):.3f}, Val Accuracy: {total_acc_val / len(val_dataset):.3f}")


# Função de Predição (predict_intent)
Usa o modelo para prever a intenção de uma frase de entrada. Retorna a intenção com maior confiança, caso seja superior ao threshold definido.

In [14]:
def predict_intent(model, text):
    with torch.no_grad():
        encoding = tokenizer(text, return_tensors='pt', padding='max_length', max_length=cfg.max_length, truncation=True)
        input_ids = encoding['input_ids'].to(device)
        attention_mask = encoding['attention_mask'].to(device)
        outputs = model(input_ids, attention_mask)
        probs = F.softmax(outputs, dim=1)
        confidence, predicted_label = torch.max(probs, dim=1)
        if confidence.item() < 0.3:
            return None
        intent = label2intent[predicted_label.item()]
        return intent


# Personalização da Resposta (personalize_response)
Substitui <HUMAN> pelo nome do usuário para personalizar a resposta.

In [15]:
def personalize_response(response, user_name):
    return response.replace("<HUMAN>", user_name)


# Funcionalidade Principal do Chatbot (main)
Gera uma interface interativa, pede o nome do usuário, permite a consulta das últimas três intenções detectadas e exibe respostas personalizadas.

In [16]:
def main():
    global model
    user_name = ""
    intents_history = deque(maxlen=3)

    print("Olá! Eu sou o seu assistente virtual.")
    if not user_name:
        user_name = input("Por favor, qual é o seu nome? ").strip()
        print(f"Prazer em conhecê-lo, {user_name}!")

    while True:
        user_input = input(f"{user_name}: ").strip()
        if user_input.lower() in ['sair', 'exit', 'quit']:
            print("Encerrando a conversa. Até logo!")
            break
        elif user_input.lower() == 'consultar intenções':
            if intents_history:
                print("Últimas três intenções detectadas:")
                for idx, intent in enumerate(list(intents_history)[::-1], 1):
                    print(f"{idx}. {intent}")
            else:
                print("Nenhuma intenção detectada até o momento.")
            continue

        intent = predict_intent(model, user_input)
        if intent:
            intents_history.append(intent)
            response = random.choice(responses[intent])
            personalized_response = personalize_response(response, user_name)
            print(f"{user_name}, sua intenção é: {intent}")
            print(f"Resposta: {personalized_response}")
        else:
            print(f"Desculpe, {user_name}, não consegui identificar sua intenção.")

if __name__ == "__main__":
    main()


Olá! Eu sou o seu assistente virtual.
Prazer em conhecê-lo, Nathan!
Nathan, sua intenção é: CourtesyGreeting
Resposta: Hi, I am great, how are you? Please tell me your GeniSys user
Nathan, sua intenção é: CourtesyGreetingResponse
Resposta: Great! Hi Nathan! How can I help?
Nathan, sua intenção é: CurrentHumanQuery
Resposta: Nathan, what can I do for you?
Últimas três intenções detectadas:
1. CurrentHumanQuery
2. CourtesyGreetingResponse
3. CourtesyGreeting
Encerrando a conversa. Até logo!


# Resultados e Casos de Teste
Resultados: O chatbot identifica intenções e responde de maneira personalizada, armazenando as últimas três intenções detectadas. Responde com o nome do usuário ao substituir <HUMAN> na resposta.

## Casos de Teste:

- Entrada: "How are you?"
- Saída Esperada: "Hello, I am great, how are you, [Nome]?"

## Respostas Aleatórias:
- Entrada: "Hi"
- Saída Esperada: "Hi, please tell me your GeniSys user."

## Consulta de Intenções:

- Entrada: "consultar intenções"
- Saída Esperada: Lista das últimas três intenções detectadas.

## Predição do Nome:
- Entrada: "What is my name?"
- Saída Esperada: "They call you [Nome], what can I do for you?"

## Encerramento:

- Entrada: "sair"
- Saída Esperada: "Encerrando a conversa. Até logo!"